In [1]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))
from binance import Client
from datetime import datetime 

import src.utils_binance as utils
import src.utils as utils2
import src.myenv as myenv
import time
myenv.telegram_key.append(utils2.get_telegram_key())

import numpy as np
import pandas as pd


<built-in method __dir__ of sqlite3.Connection object at 0x7fdd430d8240>


In [2]:
utils.get_account_balance()

New Instance of Client: <binance.client.Client object at 0x7fdd60136c80>


92.41642966

In [3]:
#client.get_account_snapshot(type='SPOT')

In [4]:
symbol = 'DYDXUSDT'
symbol_info = utils.get_client().get_symbol_info(symbol=symbol)
print(f'symbol_info: {symbol_info}')


symbol_info: {'symbol': 'DYDXUSDT', 'status': 'TRADING', 'baseAsset': 'DYDX', 'baseAssetPrecision': 8, 'quoteAsset': 'USDT', 'quotePrecision': 8, 'quoteAssetPrecision': 8, 'baseCommissionPrecision': 8, 'quoteCommissionPrecision': 8, 'orderTypes': ['LIMIT', 'LIMIT_MAKER', 'MARKET', 'STOP_LOSS_LIMIT', 'TAKE_PROFIT_LIMIT'], 'icebergAllowed': True, 'ocoAllowed': True, 'quoteOrderQtyMarketAllowed': True, 'allowTrailingStop': True, 'cancelReplaceAllowed': True, 'isSpotTradingAllowed': True, 'isMarginTradingAllowed': True, 'filters': [{'filterType': 'PRICE_FILTER', 'minPrice': '0.00100000', 'maxPrice': '10000.00000000', 'tickSize': '0.00100000'}, {'filterType': 'LOT_SIZE', 'minQty': '0.01000000', 'maxQty': '9222449.00000000', 'stepSize': '0.01000000'}, {'filterType': 'ICEBERG_PARTS', 'limit': 10}, {'filterType': 'MARKET_LOT_SIZE', 'minQty': '0.00000000', 'maxQty': '278833.65429166', 'stepSize': '0.00000000'}, {'filterType': 'TRAILING_DELTA', 'minTrailingAboveDelta': 10, 'maxTrailingAboveDelta

In [5]:
price_order = utils.get_client().get_symbol_ticker(symbol=symbol)
symbol_info, symbol_precision, quote_precision, quantity_precision, price_precision, step_size, tick_size = utils.get_symbol_info(symbol)
price = round(float(price_order['price']), symbol_precision)

take_profit, stop_loss = utils.calc_take_profit_stop_loss('LONG', price, 1.0, 1)
print((symbol_precision, price_precision, step_size, tick_size))
print((f'price: {price} - take_profit: {take_profit} - stop_loss: {stop_loss}'))

(8, 3, 0.01, 0.001)
price: 3.036 - take_profit: 3.06636 - stop_loss: 3.00564


In [6]:
amount_invested, balance = utils.get_amount_to_invest()  # ok

ledger_params = utils.get_params_operation(np.datetime64('now'), symbol, '1m', 'BUY', 1.0, amount_invested, take_profit, stop_loss,
												price, 30.0, take_profit, 0.0, 1.0, 'LONG', balance, symbol_precision, quote_precision, quantity_precision, price_precision, tick_size, step_size)  # ok
print(ledger_params)

{'operation_date': datetime.datetime(1969, 12, 31, 21, 0, 1, 703214), 'symbol': 'DYDXUSDT', 'interval': '1m', 'operation': 'BUY', 'target_margin': 1.0, 'amount_invested': 10.0, 'take_profit': 3.06636, 'stop_loss': 3.00564, 'purchase_price': 3.036, 'sell_price': 3.06636, 'pnl': 0.0, 'rsi': 30.0, 'margin_operation': 1.0, 'strategy': 'LONG', 'balance': 92.41642966, 'symbol_precision': 8, 'price_precision': 3, 'tick_size': 0.01, 'step_size': 0.001, 'quantity_precision': 2, 'quote_precision': 8}


In [7]:
#order_buy_id, order_sell_id = utils.register_operation(ledger_params)
#print((order_buy_id, order_sell_id))

In [8]:
symbol = 'DYDXUSDT'
interval = '1m'

id_buy = f'{symbol}_{interval}_buy'
id_limit = f'{symbol}_{interval}_limit'
id_stop = f'{symbol}_{interval}_stop'
orders = utils.get_client().get_all_orders(symbol=symbol)

In [9]:
purshased_price = 0.0
stop_loss = 0.0
take_profit = 0.0
df_order = pd.DataFrame(orders)
key = (df_order['clientOrderId'] == id_buy) | (df_order['clientOrderId'] == id_limit) | (df_order['clientOrderId'] == id_stop)
if True: # key.sum() > 0:
	df_order = df_order[key]
	is_purchased = df_order['status'].isin([Client.ORDER_STATUS_NEW, Client.ORDER_STATUS_PARTIALLY_FILLED, Client.ORDER_STATUS_PENDING_CANCEL]).sum() > 0
	if is_purchased:
		has_buy = df_order['clientOrderId'] == id_buy
		if has_buy.sum() > 0:
			purshased_price = float(df_order[has_buy].tail(1)['price'].values[0])
		has_limit = df_order['clientOrderId'] == id_limit
		if has_limit.sum() > 0:
			take_profit = float(df_order[has_limit].tail(1)['price'].values[0])
		has_stop = df_order['clientOrderId'] == id_stop
		if has_stop.sum() > 0:
			stop_loss = float(df_order[has_stop].tail(1)['price'].values[0])

print(f'purshased_price: {purshased_price} - stop_loss: {stop_loss} - take_profit: {take_profit}')
#df_order.info()
df_order

purshased_price: 0.0 - stop_loss: 0.0 - take_profit: 0.0


,symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,workingTime,origQuoteOrderQty,selfTradePreventionMode
24,DYDXUSDT,1133668940,-1,DYDXUSDT_1m_buy,2.90300000,3.44000000,3.44000000,9.98288000,FILLED,GTC,LIMIT,BUY,0.00000000,0.00000000,1703150713095,1703150713095,True,1703150713095,0.00000000,EXPIRE_MAKER
25,DYDXUSDT,1133669386,95811800,DYDXUSDT_1m_stop,2.38500000,3.44000000,0.00000000,0.00000000,EXPIRED,GTC,STOP_LOSS_LIMIT,SELL,2.51100000,0.00000000,1703150716417,1703183430854,False,-1,0.00000000,EXPIRE_MAKER
26,DYDXUSDT,1133669387,95811800,DYDXUSDT_1m_limit,3.03400000,3.44000000,3.44000000,10.43696000,FILLED,GTC,LIMIT_MAKER,SELL,0.00000000,0.00000000,1703150716417,1703183430854,True,1703150716417,0.00000000,EXPIRE_MAKER
36,DYDXUSDT,1134497953,-1,DYDXUSDT_1m_buy,2.97000000,3.37000000,0.00000000,0.00000000,CANCELED,GTC,LIMIT,BUY,0.00000000,0.00000000,1703186927643,1703187351431,True,1703186927643,0.00000000,EXPIRE_MAKER
37,DYDXUSDT,1134808232,-1,DYDXUSDT_1m_buy,3.02300000,3.31000000,0.00000000,0.00000000,CANCELED,GTC,LIMIT,BUY,0.00000000,0.00000000,1703204241628,1703204666296,True,1703204241628,0.00000000,EXPIRE_MAKER
38,DYDXUSDT,1134938689,-1,DYDXUSDT_1m_buy,3.05000000,3.28000000,3.28000000,10.00400000,FILLED,GTC,LIMIT,BUY,0.00000000,0.00000000,1703211664803,1703211664803,True,1703211664803,0.00000000,EXPIRE_MAKER
39,DYDXUSDT,1134939191,-1,DYDXUSDT_1m_buy,3.05000000,3.28000000,3.28000000,10.00400000,FILLED,GTC,LIMIT,BUY,0.00000000,0.00000000,1703211705343,1703211705343,True,1703211705343,0.00000000,EXPIRE_MAKER
40,DYDXUSDT,1134939369,-1,DYDXUSDT_1m_buy,3.05000000,3.28000000,3.28000000,10.00400000,FILLED,GTC,LIMIT,BUY,0.00000000,0.00000000,1703211719870,1703211719870,True,1703211719870,0.00000000,EXPIRE_MAKER
41,DYDXUSDT,1134939642,-1,DYDXUSDT_1m_buy,3.05000000,3.28000000,3.28000000,10.00400000,FILLED,GTC,LIMIT,BUY,0.00000000,0.00000000,1703211737128,1703211763740,True,1703211737128,0.00000000,EXPIRE_MAKER
42,DYDXUSDT,1134940605,-1,DYDXUSDT_1m_buy,3.05000000,3.28000000,3.28000000,10.00072000,FILLED,GTC,LIMIT,BUY,0.00000000,0.00000000,1703211778162,1703211778162,True,1703211778162,0.00000000,EXPIRE_MAKER


In [10]:
purchased, purchase_price, amount_invested, take_profit, stop_loss = utils.is_purchased(symbol, interval)

print((f'symbol: {symbol} - interval: {interval} - purchased: {purchased} - purchase_price: {purchase_price} - amount_invested: {amount_invested} - take_profit: {take_profit} - stop_loss: {stop_loss}'))

symbol: DYDXUSDT - interval: 1m - purchased: False - purchase_price: 0.0 - amount_invested: 0.0 - take_profit: 0.0 - stop_loss: 0.0


In [11]:
symbol_info, symbol_precision, quote_precision, quantity_precision, price_precision, step_size, tick_size = utils.get_symbol_info(symbol)

_time = pd.DataFrame(data=[utils.get_client().get_server_time()])				 

take_profit, stop_loss = utils.calc_take_profit_stop_loss('LONG', 3.048, 4.5, 2) 

ledger_params = utils.get_params_operation(_time['serverTime'].values[0], symbol, interval, 'BUY', 4.5, 10.0,
											take_profit, stop_loss, 3.048, 0, take_profit, 0.0, 0.0, 'LONG', balance,
											symbol_precision, quote_precision, quantity_precision, price_precision, step_size, tick_size)  # ok
print(ledger_params)

{'operation_date': datetime.datetime(1969, 12, 31, 21, 28, 23, 213698), 'symbol': 'DYDXUSDT', 'interval': '1m', 'operation': 'BUY', 'target_margin': 4.5, 'amount_invested': 10.0, 'take_profit': 3.1851599999999998, 'stop_loss': 2.77368, 'purchase_price': 3.048, 'sell_price': 3.1851599999999998, 'pnl': 0.0, 'rsi': 0, 'margin_operation': 0.0, 'strategy': 'LONG', 'balance': 92.41642966, 'symbol_precision': 8, 'price_precision': 3, 'tick_size': 0.001, 'step_size': 0.01, 'quantity_precision': 2, 'quote_precision': 8}


In [13]:
utils.register_operation(ledger_params)

Trying to register order_limit_buy: Params> {'operation_date': datetime.datetime(1969, 12, 31, 21, 28, 23, 213698), 'symbol': 'DYDXUSDT', 'interval': '1m', 'operation': 'BUY', 'target_margin': 4.5, 'amount_invested': 10.0, 'take_profit': 3.1851599999999998, 'stop_loss': 2.77368, 'purchase_price': 3.048, 'sell_price': 3.1851599999999998, 'pnl': 0.0, 'rsi': 0, 'margin_operation': 0.0, 'strategy': 'LONG', 'balance': 92.41642966, 'symbol_precision': 8, 'price_precision': 3, 'tick_size': 0.001, 'step_size': 0.01, 'quantity_precision': 2, 'quote_precision': 8, 'quantity': 3.28}
ORDER BUY: {'symbol': 'DYDXUSDT', 'quantity': 3.28, 'price': '3.048', 'newClientOrderId': 'DYDXUSDT_1m_buy'}
order_buy_id: None
Trying to register order_oco_sell: Params> {'operation_date': datetime.datetime(1969, 12, 31, 21, 28, 23, 213698), 'symbol': 'DYDXUSDT', 'interval': '1m', 'operation': 'BUY', 'target_margin': 4.5, 'amount_invested': 10.0, 'take_profit': 3.1851599999999998, 'stop_loss': 2.77368, 'purchase_pric

(None,
 {'orderListId': 95852693,
  'contingencyType': 'OCO',
  'listStatusType': 'EXEC_STARTED',
  'listOrderStatus': 'EXECUTING',
  'listClientOrderId': 'RnmqUaWr6MQu3ntXMvGE39',
  'transactionTime': 1703213990008,
  'symbol': 'DYDXUSDT',
  'orders': [{'symbol': 'DYDXUSDT',
    'orderId': 1134988235,
    'clientOrderId': 'DYDXUSDT_1m_stop'},
   {'symbol': 'DYDXUSDT',
    'orderId': 1134988236,
    'clientOrderId': 'DYDXUSDT_1m_limit'}],
  'orderReports': [{'symbol': 'DYDXUSDT',
    'orderId': 1134988235,
    'orderListId': 95852693,
    'clientOrderId': 'DYDXUSDT_1m_stop',
    'transactTime': 1703213990008,
    'price': '2.63500000',
    'origQty': '22.93000000',
    'executedQty': '0.00000000',
    'cummulativeQuoteQty': '0.00000000',
    'status': 'NEW',
    'timeInForce': 'GTC',
    'type': 'STOP_LOSS_LIMIT',
    'side': 'SELL',
    'stopPrice': '2.77400000',
    'workingTime': -1,
    'selfTradePreventionMode': 'EXPIRE_MAKER'},
   {'symbol': 'DYDXUSDT',
    'orderId': 1134988236,